# Number of sample paths
This notebook illustrates average number of sample paths for relative and absolute tolerance approximation.

In [1]:
import sys
sys.path.append('..')  

from relu_nets import ReLUNet  
from hyperbox import Hyperbox  
from lipMIP import LipMIP
from helper_functions_demo import *
from other_methods import StochasticApproximation, StochasticApproximationEqDiv, StochasticApproximationUCBDynamic

In [2]:
def get_trained_network(layer_sizes, rad, dim):
    test_network = ReLUNet(layer_sizes)
    num_points = 1000
    X, y = get_random_dataset(num_points=num_points, radius=rad, num_lakes=3, dim=dim)
    train_X = X[:int(num_points * 0.8)]
    train_y = y[:int(num_points * 0.8)].unsqueeze(1)
    # test_X = X[int(num_points * 0.8):]
    # test_y = y[int(num_points * 0.8):].unsqueeze(1)

    loss_fn = torch.nn.MSELoss()
    optimizer = torch.optim.Adam(test_network.parameters(), lr=5e-4)

    epochs = 500
    for epoch in range(epochs):
        test_network.train()
        pred = test_network(train_X)
        loss = loss_fn(pred, train_y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # if epoch % 5 == 0:
        #     test_network.eval()
        #     pred = test_network(test_X)
        #     loss = loss_fn(pred, test_y)
        #     print(f"Epoch {epoch}: Loss: {loss.item():.4f}")
        
    return test_network

In [3]:
def is_in_tolerance_range(value, exact, tol, mode):
    if mode == "Absolute":
        if torch.abs(exact - value) <= tol:
            return True
    else:
        if torch.abs(exact - value)/value*100.0 <= tol:
            return True
    return False

def run_sample_paths_experiment(dim, rad, tol, mode, domain, c_vector, num_networks, width, num_samples, max_iter, divisions_per_dimension, c, partition_step, verbose=False):

    infeasible = 0
    hidden_layers = 1
    it_cnt = np.zeros((3, num_networks, num_samples))
    failed = [0, 0, 0]
    
    # for _ in tqdm.tqdm(range(num_networks)):
    for _ in range(num_networks):
        for j in range(num_samples):
            if verbose: print(f"layers = {layer_sizes}, j = {j}")
            layer_sizes = [dim] + [width] * hidden_layers + [1]
            test_network = get_trained_network(layer_sizes, rad=rad, dim=dim)
            simple_prob = LipMIP(test_network, domain, c_vector, verbose=False, num_threads=5)
            simple_result = simple_prob.compute_max_lipschitz()
            if simple_result is None:
                infeasible += 1
                continue
            LipMIP_result = torch.tensor(simple_result.as_dict()["value"])
            
            SA = StochasticApproximation(test_network, c_vector, domain)
            res = SA.compute(v=False, max_iter=max_iter, exact=LipMIP_result, tol=tol, mode=mode)
            it_cnt[0, hidden_layers-1, j] = SA.iteration_count
            if not is_in_tolerance_range(res, LipMIP_result, tol, mode):
                failed[0] += 1
                
            SA_eq_div = StochasticApproximationEqDiv(network=test_network, c_vector=c_vector, domain=domain, divisions_per_dimension=divisions_per_dimension)
            res = SA_eq_div.compute(v=False, total_iterations=max_iter, exact=LipMIP_result, tol=tol, mode=mode)
            it_cnt[1, hidden_layers-1, j] = SA_eq_div.iteration_count
            if not is_in_tolerance_range(res, LipMIP_result, tol, mode):
                failed[1] += 1
            
            SA_UCB_Dynamic = StochasticApproximationUCBDynamic(test_network, c_vector, domain, c, partition_step)
            res = SA_UCB_Dynamic.compute(v=False, max_iter=max_iter, exact=LipMIP_result, tol=tol, mode=mode)
            it_cnt[2, hidden_layers-1, j] = SA_UCB_Dynamic.iteration_count
            if not is_in_tolerance_range(res, LipMIP_result, tol, mode):
                failed[2] += 1
        
        ### PLOT AFTER EACH LAYER INCREASE >>>>
        
        plot_results(it_cnt=it_cnt[:, :hidden_layers], dim=DIMENSION, tol=tol, mode=mode, num_networks=hidden_layers, width=width, failed=failed, all_number=num_samples * hidden_layers)
        
        ### <<<<<
        
        hidden_layers += 1
        
    print(f"Infeasible = {infeasible}")
    return it_cnt

In [4]:
def plot_results(it_cnt, dim, tol, mode, num_networks, width, failed, all_number):
    num_hidden_layers = list(range(1, num_networks + 1))
    if failed[0] != all_number:
        plt.errorbar(num_hidden_layers, it_cnt[0].mean(axis=1), yerr=it_cnt[0].std(axis=1), fmt='o-', capsize=5, label="Mean Iterations (Original)")
    if failed[1] != all_number:
        plt.errorbar(num_hidden_layers, it_cnt[1].mean(axis=1), yerr=it_cnt[1].std(axis=1), fmt='D-', capsize=5, label="Mean Iterations (Static partitioning)")
    if failed[2] != all_number:
        plt.errorbar(num_hidden_layers, it_cnt[2].mean(axis=1), yerr=it_cnt[2].std(axis=1), fmt='D-', capsize=5, label="Mean Iterations (UCB, Dynamic partitioning)")
    plt.xlabel(f'Hidden layers (width = {width})')
    plt.ylabel('Sample paths')
    # plt.yscale('log')
    plt.legend()
    
    plt.grid()
    plt.title(f'Number of sample paths for approximation with {mode} Tolerance = {tol}{"%" if mode == "Relative" else ""}\n'
              f'Input dimension = {dim}\n'
              f'Sample paths limit exceeded (No partitioning): {failed[0]}/{all_number} ({failed[0]/all_number*100:.2f}%)\n'
              f'Limit exceeded (Static partitioning): {failed[1]}/{all_number} ({failed[1]/all_number*100:.2f}%)\n'
              f'Limit exceeded (UCB, Dynamic partitioning): {failed[2]}/{all_number} ({failed[2]/all_number*100:.2f}%)')
    plt.show()

In [6]:
DIMENSION = 7
radius = 2.0
domain = Hyperbox.build_custom_hypercube(DIMENSION, center=0.0, radius=radius)
c_vector = torch.tensor([1.0])

tol = 5.0
num_networks = 4
width = 8
num_samples = 30
max_iter = 20000
mode = "Relative"
it_count = run_sample_paths_experiment(dim=DIMENSION, tol=tol, mode=mode, domain=domain, c_vector=c_vector, num_networks=num_networks, width=width, num_samples=num_samples, max_iter=max_iter, verbose=False, divisions_per_dimension=2, c=10, partition_step=2, rad=radius)

In [ ]:
failed = [0, 0, 0]
failed[0] = 104
failed[1] = 108
failed[2] = 85
plot_results(it_cnt=it_count, dim=DIMENSION, tol=tol, mode=mode, num_networks=num_networks, width=width, failed=failed, all_number=num_samples * num_networks)